<a href="https://colab.research.google.com/github/vapaul10/Portfolio-of-Valeri/blob/main/Copy_of_Amazon_Book_Order.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Stratascratch tutorial

### Monthly Sales Rolling Average

###### Question 
You have been asked to calculate the rolling average for book sales in 2022.


A rolling average continuously updates a data set's average to include all data in the set up to that point. For example, the rolling average for February would be calculated by adding the book sales from January and February and dividing by two; the rolling average for March would be calculated by adding the book sales from January, February, and March and dividing by three; and so on.


Output the date, the sales for that date, and an extra column containing the rolling average rounded to the nearest whole number.

In [ ]:
#install Python Datatable (same at data.table() in R)
!pip install datatable
import datatable as dt

from datatable import (dt, f, by, ifelse, update, sort, count, min, max, mean, sum, rowsum)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Import your libraries
import pandas as pd
import numpy as np
import datetime as datetimes
import time as time

In [ ]:
# Install data from Amazon Books database Start writing code
amazon_books_pd = dt.fread("https://raw.githubusercontent.com/vapaul10/Portfolio-of-Valeri/41cefe2cd5218ab780ee4d3649a51b0fbf0f9947/amazon_books.csv")
#amazon_books_pd = pd.read_csv("https://raw.githubusercontent.com/vapaul10/Portfolio-of-Valeri/41cefe2cd5218ab780ee4d3649a51b0fbf0f9947/amazon_books.csv")

amazon_books_pd.head(10)

,book_id,book_title,unit_price
,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,B001,The Hunger Games,25
1,B002,The Outsiders,50
2,B003,To Kill a Mockingbird,100
3,B004,Pride and Prejudice,20
4,B005,Twilight,30
5,B006,The Book Thief,50
6,B007,Animal Farm,40
7,B008,The Chronicles of Narnia,30
8,B009,Fahrenheit 451,50


In [ ]:
## Data Attributes
amazon_books_pd.names

('book_id', 'book_title', 'unit_price')

In [ ]:
## Install data from Book_Orders database. Dt  Start writing code
book_orders_pd = dt.fread("https://raw.githubusercontent.com/vapaul10/Portfolio-of-Valeri/main/book_orders.csv")
#book_orders_pd = pd.read_csv("https://raw.githubusercontent.com/vapaul10/Portfolio-of-Valeri/main/book_orders.csv")

book_orders_pd.head()

,order_id,order_date,book_id,quantity
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,1001,1/10/2022,B001,1
1,1002,1/10/2022,B009,1
2,1003,1/15/2022,B012,2
3,1004,2/4/2022,B006,1
4,1005,2/4/2022,B019,2
5,1006,2/5/2022,B017,1
6,1009,3/1/2022,B005,1
7,1010,3/5/2022,B001,1
8,1011,3/10/2022,B002,2


In [ ]:
# Amazon Books in Datatable frame

In [ ]:
amazon_books_p=dt.Frame(amazon_books_pd)

In [ ]:
amazon_books_pd = amazon_books_p.to_pandas()
amazon_books_pd.head()

,book_id,book_title,unit_price
0,B001,The Hunger Games,25
1,B002,The Outsiders,50
2,B003,To Kill a Mockingbird,100
3,B004,Pride and Prejudice,20
4,B005,Twilight,30


In [ ]:
book_orders_p=dt.Frame(book_orders_pd)

In [ ]:
book_orders_pd = book_orders_p.to_pandas()
book_orders_pd.head()

,order_id,order_date,book_id,quantity
0,1001.0,1/10/2022,B001,1.0
1,1002.0,1/10/2022,B009,1.0
2,1003.0,1/15/2022,B012,2.0
3,1004.0,2/4/2022,B006,1.0
4,1005.0,2/4/2022,B019,2.0


EXPLORATORY DATA ANALYSIS

###### First line converts the Amazon_Books Datatable object to Pandas object.
###### Then a summary of books is generated.

In [ ]:
amazon_books_pd=amazon_books_p.to_pandas()  #Convert the Datatable object to Pandas object 
books_summary = amazon_books_pd.describe()
books_summary = books_summary.transpose()
books_summary()

,count,mean,std,min,25%,50%,75%,max
unit_price,20.0,49.25,22.95734,20.0,30.0,47.5,60.0,100.0


##### First line converts the Book_Orders Datatable object to Pandas object.

##### Then a summary of orders is generated

In [ ]:
book_orders_pd=book_orders_p.to_pandas()  #Convert the Datatable object to Pandas object 
order_summary = book_orders_pd.describe()
order_summary = order_summary.transpose()
order_summary

,count,mean,std,min,25%,50%,75%,max
order_id,74.0,1043.986486,25.075499,1001.0,1022.25,1045.5,1065.75,1084.0
quantity,74.0,1.540541,0.705928,1.0,1.00,1.0,2.00,4.0


In [ ]:
#Factor the features of Amazon Books and Book Order databases. 
df = pd.merge(amazon_books_pd, book_orders_pd, on='book_id', how='inner').assign(sales =lambda x: x['unit_price']*x['quantity'])
df = df[pd.to_datetime(df['order_date']).dt.to_period('M').dt.year == 2022]
df['order_date'] = df['order_date'].dt
df = df.groupby('order_date')['sales'].sum().reset_index().sort_values('order_date').assign(rolling_avg = lambda x: x['sales'].expanding().mean().round(0))
df.head()

,order_date,sales,rolling_avg
0,1/10/2022,75.0,75.0
1,1/15/2022,70.0,72.0
2,10/10/2022,30.0,58.0
3,10/5/2022,80.0,64.0
4,10/6/2022,120.0,75.0


In [ ]:
### Merge Data

In [ ]:
# merge data
merged = pd.merge(book_orders_pd, amazon_books_pd, on="book_id", how="inner")
merged["order_date"] = pd.to_datetime(merged["order_date"])
merged["order_month"] = merged["order_date"].dt.month
merged["year"] = merged["order_date"].dt.year
merged["sales"] = (merged["unit_price"]*merged["quantity"])
merged = merged.loc[(merged["year"] == 2022), :]
merged.head(15)


,order_id,order_date,book_id,quantity,book_title,unit_price,order_month,year,sales
0,1001.0,2022-01-10,B001,1.0,The Hunger Games,25,1,2022,25.0
1,1010.0,2022-03-05,B001,1.0,The Hunger Games,25,3,2022,25.0
2,1020.0,2022-04-25,B001,2.0,The Hunger Games,25,4,2022,50.0
3,1028.0,2022-06-10,B001,1.0,The Hunger Games,25,6,2022,25.0
4,1037.0,2022-07-05,B001,1.0,The Hunger Games,25,7,2022,25.0
5,1055.0,2022-09-10,B001,1.0,The Hunger Games,25,9,2022,25.0
6,1064.0,2022-11-05,B001,1.0,The Hunger Games,25,11,2022,25.0
7,1074.0,2022-12-05,B001,2.0,The Hunger Games,25,12,2022,50.0
8,1002.0,2022-01-10,B009,1.0,Fahrenheit 451,50,1,2022,50.0
9,1029.0,2022-06-10,B009,1.0,Fahrenheit 451,50,6,2022,50.0


In [ ]:
### Result
result = (merged.groupby("order_month")["sales"]).sum().to_frame("monthly_sales").sort_values(by = "order_month").reset_index()
result["rolling_average"] = result["monthly_sales"].expanding().mean().round(0)
result

,order_month,monthly_sales,rolling_average
0,1,145.0,145.0
1,2,250.0,198.0
2,3,315.0,237.0
3,4,400.0,278.0
4,5,450.0,312.0
5,6,315.0,312.0
6,7,465.0,334.0
7,8,540.0,360.0
8,9,315.0,355.0
9,10,350.0,354.0
